## 분석 전략

시도 1 /
종분류, 3차원직선을 도입한다.-> 0.17201

시도 2 /
데이터가 적은게 문제가 된다고 본다. 데이터가 적어서 발생하는것은 잘못된 방향으로 적합되는것일테니 과적합이 문제일것이다. 따라서 이를 해결하는 정규화 툴을 넣어보겠다.
-> 0.1717

시도 3 /
왠지 잘 되는거 같으니 정규화를 좀더 강하게 시켜보자. -> 0.1723 

시도 4/
너무 과도한 정규화로 학습이 잘 이뤄지지 못했다고 판단된다. -> 0.17151

시도 5/
정규화로 극적인 변화를 줄 수 없었다. MAE 방식에 유리하다는 다른 회귀 모델을 사용한다.


## 코드


### 데이터 전처리

In [1]:
# 라이브러리
import pandas as pd
import numpy as np

In [2]:
# 데이터 불러오기
train = pd.read_csv('data/iris_train.csv')
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


In [3]:
# 품종에 따라 train set 3개로 나누기
virginica = train[train['species'] == 'virginica'] # 전체 붓꽃 데이터에서 virginica만 추출
setosa = train[train['species'] == 'setosa'] # 전체 붓꽃 데이터에서 setosa만 추출
versicolor = train[train['species'] == 'versicolor'] # 전체 붓꽃 데이터에서 virsicolor만 추출

In [4]:
virginica.head() # 확인

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
5,5,virginica,6.5,5.8,3.0,2.2
11,11,virginica,6.3,4.9,2.7,1.8
15,15,virginica,7.6,6.6,3.0,2.1


In [5]:
# x값, y값으로 나누기 (혹시 아래 방법을 더 쉽게 할 수 있다면 댓글 부탁드려요)

# virginica
x_train_virginica = virginica[['sepal length (cm)', 'petal length (cm)']]
y_train_virginica_sepal = virginica[['sepal width (cm)']]
y_train_virginica_petal = virginica[['petal width (cm)']]

# setosa
x_train_setosa = setosa[['sepal length (cm)', 'petal length (cm)']]
y_train_setosa_sepal = setosa[['sepal width (cm)']]
y_train_setosa_petal = setosa[['petal width (cm)']]

# versicolor
x_train_versicolor = versicolor[['sepal length (cm)', 'petal length (cm)']]
y_train_versicolor_sepal = versicolor[['sepal width (cm)']]
y_train_versicolor_petal = versicolor[['petal width (cm)']]

In [6]:
x_train_virginica.head() # 확인

,sepal length (cm),petal length (cm)
2,6.2,4.8
3,7.2,6.1
5,6.5,5.8
11,6.3,4.9
15,7.6,6.6


In [7]:
y_train_virginica_sepal.head() # 확인

,sepal width (cm)
2,2.8
3,3.6
5,3.0
11,2.7
15,3.0


### 모델 만들기

3차원 공간에서 직선을 만드려면 `y = a * x1 + b * x2 + c` 형태가 되어야 한다.
또한 점(데이터 값)과 직선(모델)과의 최소 길이를 구하는 수식이 복잡하다.

좋은 패키지가 없나 찾아보는 과정에서 `사이킷런`과 `다중선형회귀`라는 키워드를 알게되어 이를 이용해서 최적화를 시도한다.

In [8]:
# 라이브러리
from sklearn.linear_model import RANSACRegressor, Ridge

In [9]:
# 모델 선언 (객체 선언)

# virginica
lr_virginica_sepal = RANSACRegressor(random_state=0)
lr_virginica_petal = Ridge(alpha=0.1)

# setosa
lr_setosa_sepal = RANSACRegressor(random_state=0)
lr_setosa_petal = Ridge(alpha=0.1)

# versicolor
lr_versicolor_sepal = Ridge(alpha=0.1)
lr_versicolor_petal = Ridge(alpha=0.1)

### 모델 학습

In [10]:
# 모델 학습 (아래 형태로 넣으면 알아서 학습이 된다.)

# virginica
lr_virginica_sepal.fit(x_train_virginica, y_train_virginica_sepal)
lr_virginica_petal.fit(x_train_virginica, y_train_virginica_petal)

# setosa
lr_setosa_sepal.fit(x_train_setosa, y_train_setosa_sepal)
lr_setosa_petal.fit(x_train_setosa, y_train_setosa_petal)

# versicolor
lr_versicolor_sepal.fit(x_train_versicolor, y_train_versicolor_sepal)
lr_versicolor_petal.fit(x_train_versicolor, y_train_versicolor_petal)

Ridge(alpha=0.1)

### 학습된 모델로 결과 평가하기

In [11]:
# 결과값 계산

# virginica
y_predict_virginica_sepal = lr_virginica_sepal.predict(x_train_virginica)
y_predict_virginica_petal = lr_virginica_petal.predict(x_train_virginica)

# setosa
y_predict_setosa_sepal = lr_setosa_sepal.predict(x_train_setosa)
y_predict_setosa_petal = lr_setosa_petal.predict(x_train_setosa)

# versicolor
y_predict_versicolor_sepal = lr_versicolor_sepal.predict(x_train_versicolor)
y_predict_versicolor_petal = lr_versicolor_petal.predict(x_train_versicolor)

In [12]:
# 대회 규칙 탭의 MAE() 함수를 그대로 사용

def MAE(true, pred):
    score = np.mean(np.abs(true.to_numpy()-pred))
    return score

# 정답과 예측 값을 함수에 넣어 결과를 확인

# virginica
mean_error = MAE(y_train_virginica_sepal, y_predict_virginica_sepal)
print(f"virginica_sepal의 평균 에러는 {mean_error:.2f} 입니다.")
mean_error = MAE(y_train_virginica_petal, y_predict_virginica_petal)
print(f"virginica_petal의 평균 에러는 {mean_error:.2f} 입니다.")

# setosa
mean_error = MAE(y_train_setosa_sepal, y_predict_setosa_sepal)
print(f"setosa_sepal의 평균 에러는 {mean_error:.2f} 입니다.")
mean_error = MAE(y_train_setosa_petal, y_predict_setosa_petal)
print(f"setosa_petal의 평균 에러는 {mean_error:.2f} 입니다.")

# versicolor
mean_error = MAE(y_train_versicolor_sepal, y_predict_versicolor_sepal)
print(f"versicolor_sepal의 평균 에러는 {mean_error:.2f} 입니다.")
mean_error = MAE(y_train_versicolor_petal, y_predict_versicolor_petal)
print(f"versicolor_petal의 평균 에러는 {mean_error:.2f} 입니다.")

virginica_sepal의 평균 에러는 0.23 입니다.
virginica_petal의 평균 에러는 0.18 입니다.
setosa_sepal의 평균 에러는 0.13 입니다.
setosa_petal의 평균 에러는 0.04 입니다.
versicolor_sepal의 평균 에러는 0.20 입니다.
versicolor_petal의 평균 에러는 0.11 입니다.


### test_data 예측하기

In [13]:
# 예측을 위한 데이터 불러오기

test = pd.read_csv('data/iris_test.csv')
test.head() # 확인

,id,species,sepal length (cm),petal length (cm)
0,0,setosa,5.4,1.7
1,1,setosa,5.7,1.5
2,2,setosa,5.3,1.5
3,3,setosa,5.1,1.9
4,4,virginica,6.0,4.8


In [14]:
# 종별로 계산 수행하기

# 결과 저장 리스트에 저장하기 
#(np, pd의 사용이 어려워 파이썬스럽게 다뤘습니다. 좋은 방법을 아시면 댓글 부탁드립니다.)
predict_sepal_width = []
predict_petal_width = []

for idx, row in test.iterrows():
    if row['species'] == 'virginica':
        predict_sepal_width.extend(lr_virginica_sepal.predict([[row['sepal length (cm)'], row['petal length (cm)']]]).tolist()[0])
        predict_petal_width.extend(lr_virginica_petal.predict([[row['sepal length (cm)'], row['petal length (cm)']]]).tolist()[0])
    elif row['species'] == 'setosa':
        predict_sepal_width.extend(lr_setosa_sepal.predict([[row['sepal length (cm)'], row['petal length (cm)']]]).tolist()[0])
        predict_petal_width.extend(lr_setosa_petal.predict([[row['sepal length (cm)'], row['petal length (cm)']]]).tolist()[0])
    elif row['species'] == 'versicolor':
        predict_sepal_width.extend(lr_versicolor_sepal.predict([[row['sepal length (cm)'], row['petal length (cm)']]]).tolist()[0])
        predict_petal_width.extend(lr_versicolor_petal.predict([[row['sepal length (cm)'], row['petal length (cm)']]]).tolist()[0])

predict_petal_width

c:\users\yc yoon\desktop\githup\dacon\venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but RANSACRegressor was fitted with feature names
  warnings.warn(
c:\users\yc yoon\desktop\githup\dacon\venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
c:\users\yc yoon\desktop\githup\dacon\venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but RANSACRegressor was fitted with feature names
  warnings.warn(
c:\users\yc yoon\desktop\githup\dacon\venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
c:\users\yc yoon\desktop\githup\dacon\venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but RANSACRegressor was fitted with feature names
  warnings.warn(
c:\users\yc yoon\desktop\

[0.2725593589819124,
 0.25112237082425026,
 0.23235138160608027,
 0.29399634713957457,
 1.838232524151969,
 1.984974629581573,
 0.23704412891062276,
 0.2005155246568079,
 0.21358039238791027,
 1.4245618380559923,
 0.1958227773522654,
 0.1911300300477229,
 1.8846480427781378,
 2.0255860047428085,
 1.331209123612564,
 1.8491108858753946,
 0.18174453543863783,
 1.7642561386140156,
 1.4578884459159283,
 2.1024644620132285,
 2.0422685099311675,
 0.27991810522594707,
 1.284628297479868,
 0.24072350203264015,
 1.1380676477673641,
 0.22664526011901254,
 0.2584811170682849,
 0.20990101926589289,
 0.20152889883933311,
 1.8708676093222458,
 1.1980937543508563,
 0.22765863430153777,
 0.154601425793908,
 2.068618711196649,
 1.8800545649595073,
 0.22296588699699527,
 0.1592941730984505,
 1.3245820144761842,
 0.23603075472809765,
 1.458079508093965,
 0.22296588699699527,
 0.18275790962116306,
 2.0193011208380134,
 0.23603075472809765,
 1.9591051687559522,
 0.11071407529605848,
 2.0113248308470553,
 1

### 제출용 파일 만들기

In [15]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('data/sample_submission.csv')
submission.head()

,id,sepal width (cm),petal width (cm)
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0


In [16]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['sepal width (cm)'] = predict_sepal_width
submission['petal width (cm)'] = predict_petal_width

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,sepal width (cm),petal width (cm)
0,0,3.746998,0.272559
1,1,3.955660,0.251122
2,2,3.660739,0.232351
3,3,3.538336,0.293996
4,4,2.910615,1.838233
...,...,...,...
70,70,2.663716,1.238047
71,71,2.984637,0.154601
72,72,2.786197,2.062334
73,73,2.306814,0.884900


In [18]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
submission.to_csv("211028_submission_3.csv", index=False)

## 후기

numpy와 pandas의 사용이 어려워 원하는 결과가 나오면 다른 방법을 고민하지 않고 바로 넘어갔습니다. 따라서 더 최적(속도나 출력이나)의 방법이 있을 수 있습니다. 